In [1]:
import sys
import pathlib
sys.path.append('../../../build')
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/03-motion/solutions'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/03-motion/solutions'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../../scenarios/pandasTable.g") 
D = C.view()

obj = C.addFrame("object")
obj.setPosition([.8,.2,1.])
obj.setQuaternion([1,0.76,0.1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

# let's add an obstacle
obs = C.addFrame("obstacle")
obs.setContact(1)
# obs.setPosition([.5,0,1.])
obs.setPosition([.45,.1,1.2])
obs.setQuaternion([1,0,0,0])
obs.setShape(ry.ST.sphere, [.1])
obs.setColor([1,1,0])
D = C.view()

In [3]:
mode = 0 # 0 is normal graps, 1 is touch
collisions = True # True or False
komo = C.komo_path(1.,40, 5., True)

In [ ]:
margin = -.0 # -.01
if mode == 0:
    komo.addObjective([0.7, 1.], 
                    ry.FS.scalarProductXZ, 
                    ["R_gripperCenter", "object"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.])
    
    komo.addObjective([1.], 
                      ry.FS.positionDiff, 
                      ["R_gripperCenter", "object"], 
                      ry.OT.eq, 
                      [1e2])

    komo.addObjective([0.7, 1.0], 
                      ry.FS.positionRel, 
                      ["object", "R_gripperCenter"], 
                      # ry.OT.sos, 
                      ry.OT.eq,
                      [1e2], 
                      [0,0,-0.1], 
                      order=2)

    komo.addObjective([0.7, 1.], 
                    ry.FS.scalarProductZZ, 
                    ["R_gripperCenter", "object"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.])
elif mode == 1:
        komo.addObjective([1, 1.], 
                    ry.FS.distance, 
                    ["R_finger1_coll", "object"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.])
    
if collisions:
    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_gripper_coll", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [0.])
    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_finger1_coll", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [margin])
    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_finger2_coll", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [margin])
        
    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_panda_coll5", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [margin])

    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_panda_coll6", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [margin])

    komo.addObjective([0, 1.], 
                    ry.FS.distance, 
                    ["R_panda_coll7", "obstacle"], 
                    ry.OT.ineq, 
                    [1e2],
                    [margin])




komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.optimize()
komo.getReport()

In [8]:
V = komo.view_play(True, 0.1)

In [9]:
VV = komo.view(True,"")

In [11]:
V = komo.view_play(True, 0.3)

In [12]:
for i in range(40):
    C.setFrameState(komo.getFrameState(i))
    D = C.view()
    input()


KeyboardInterrupt: Interrupted by user